In [1]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor, as_completed
import itertools
from tqdm.notebook import tqdm  # This version works well in Jupyter notebooks
import logging
import os

Importing plotly failed. Interactive plots will not work.


In [2]:
def connect_and_run_query(query: str, credentials_path: str = '/Users/paz/Documents/github/auth/zwickfi-app-service-account.json') -> pd.DataFrame:
    try:
        # Load credentials from the service account key file
        creds = service_account.Credentials.from_service_account_file(credentials_path)

        # Create a BigQuery client using the service account credentials
        client = bigquery.Client(credentials=creds)

        # Run the query
        query_job = client.query(query)

        # Wait for the job to finish and return results as a DataFrame
        results = query_job.result().to_dataframe()

        return results

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [3]:
# Example usage
query = """
    select * from analytics.credit_card_spending_for_forecast where account_id = '169377965945838866'
"""

# Path to your service account key file
service_account_key_path = '/Users/paz/Documents/github/auth/zwickfi-app-service-account.json'

df = connect_and_run_query(query, service_account_key_path)

df.head()

/Users/paz/Documents/github/zwickfi-monarch/.venv/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,surrogate_pk,account_id,account_name,credit_card_due_date,statement_start_date,statement_end_date,due_month,amount
0,0b0d01b9076ea883a9b671afb089143b,169377965945838866,Blue Cash Preferred® (...3006),2024-11-04,2024-09-09,2024-10-08,2024-11-01,5216.59
1,f5e850cad0c7c1cf2705edc7ec97d951,169377965945838866,Blue Cash Preferred® (...3006),2024-10-03,2024-08-09,2024-09-08,2024-10-01,8341.81
2,6e767e24fe360a6008ef57841fbc0105,169377965945838866,Blue Cash Preferred® (...3006),2024-09-03,2024-07-09,2024-08-08,2024-09-01,7278.27
3,94170b6cd33373da996fa0b2ac67875b,169377965945838866,Blue Cash Preferred® (...3006),2024-08-05,2024-06-09,2024-07-08,2024-08-01,5894.92
4,86aa6356875688cabd222dde4f09ec1f,169377965945838866,Blue Cash Preferred® (...3006),2024-07-03,2024-05-09,2024-06-08,2024-07-01,6233.52


In [4]:
def forecast_amount(df: pd.DataFrame, periods: int = 24,
                    growth: str = 'linear',
                    changepoints: list = None,
                    n_changepoints: int = 25,
                    changepoint_range: float = 0.8,
                    yearly_seasonality: bool = 'auto',
                    weekly_seasonality: bool = 'auto',
                    daily_seasonality: bool = 'auto',
                    holidays: pd.DataFrame = None,
                    seasonality_mode: str = 'additive',
                    seasonality_prior_scale: float = 10.0,
                    holidays_prior_scale: float = 10.0,
                    changepoint_prior_scale: float = 0.05,
                    uncertainty_samples: int = 1000,
                    add_monthly_seasonality: bool = False,
                    yearly_fourier_order: int = 10,
                    monthly_fourier_order: int = 5) -> pd.DataFrame:
    
    # Step 1: Prepare the DataFrame
    df['due_month'] = pd.to_datetime(df['due_month'])
    df = df.rename(columns={'due_month': 'ds', 'amount': 'y'})
    
    # Step 2: Initialize the Prophet model with all hyperparameters
    model = Prophet(growth=growth,
                    changepoints=changepoints,
                    n_changepoints=n_changepoints,
                    changepoint_range=changepoint_range,
                    yearly_seasonality=False,  # Set to False and add manually
                    weekly_seasonality=weekly_seasonality,
                    daily_seasonality=daily_seasonality,
                    holidays=holidays,
                    seasonality_mode=seasonality_mode,
                    seasonality_prior_scale=seasonality_prior_scale,
                    holidays_prior_scale=holidays_prior_scale,
                    changepoint_prior_scale=changepoint_prior_scale,
                    uncertainty_samples=uncertainty_samples)

    # Add yearly seasonality with customizable Fourier order
    if yearly_seasonality:
        model.add_seasonality(name='yearly', period=365.25, fourier_order=yearly_fourier_order)
    
    # Add custom monthly seasonality if specified
    if add_monthly_seasonality:
        model.add_seasonality(name='monthly', period=30.5, fourier_order=monthly_fourier_order)

    # Step 3: Fit the model with historical data
    model.fit(df)

    # Step 4: Create a DataFrame for future dates
    future_dates = model.make_future_dataframe(periods=periods, freq='MS')

    # Step 5: Make the forecast
    forecast = model.predict(future_dates)

    # Step 6: Return the forecasted DataFrame
    return forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], model

def forecast_cross_validation(df: pd.DataFrame, model: Prophet, 
                              horizon: str = '365 days', initial: str = '730 days', 
                              period: str = '180 days') -> pd.DataFrame:
    
    # Step 1: Perform cross-validation using the same model instance
    df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
    
    # Step 2: Calculate performance metrics
    df_metrics = performance_metrics(df_cv)

    # Step 3: Plot Cross-Validation Results (Optional)
    # model.plot(df_cv)
    # plt.title('Cross-Validation Forecast vs Actual')
    # plt.show()

    return df_cv, df_metrics

In [5]:
forecast_df, model = forecast_amount(df)
df_cv, df_metrics = forecast_cross_validation(df, model, horizon='365 days', initial='730 days', period='180 days')

21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing


  0%|          | 0/18 [00:00<?, ?it/s]

21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:23 - cmdstanpy - INFO - Chain [1] done processing
21:39:23 - cmdstanpy - INFO - Chain [1] start processing
21:39:24 - cmdstanpy - INFO - Chain [1] done processing
21:39:24 - cmdstanpy - INFO - Chain [1] start processing
21:39:24 - cmdstanpy - INFO - Chain [1] done processing
21:39:24 - cmdstanpy - INFO - Chain [1] start processing
21:39:24 - cmdstanpy - INFO - Chain [1]

In [6]:
df_metrics.to_clipboard()

In [7]:
# Function to run each model configuration
def run_prophet_model(df, params, periods):
    try:
        # Unpack hyperparameters
        (changepoint_prior_scale, seasonality_mode, seasonality_prior_scale, n_changepoints,
         yearly_seasonality, monthly_seasonality, uncertainty_samples,
         yearly_fourier_order, monthly_fourier_order) = params

        # Call forecast_amount to create and fit the model
        forecast_df, model = forecast_amount(
            df=df,
            periods=periods,
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_mode=seasonality_mode,
            seasonality_prior_scale=seasonality_prior_scale,
            n_changepoints=n_changepoints,
            yearly_seasonality=yearly_seasonality,
            uncertainty_samples=uncertainty_samples,
            add_monthly_seasonality=monthly_seasonality,
            yearly_fourier_order=yearly_fourier_order,
            monthly_fourier_order=monthly_fourier_order
        )

        # Perform cross-validation using the same model
        df_cv, df_metrics = forecast_cross_validation(df, model)

        # Collect the mean RMSE for this set of hyperparameters
        rmse = df_metrics['rmse'].mean()
        mape = df_metrics['mape'].mean()
        coverage = df_metrics['coverage'].mean()

        return {
            'params': params,
            'rmse': rmse,
            'mape': mape,
            'coverage': coverage
        }
    except Exception as e:
        print(f"An error occurred with params {params}: {e}")
        return None

# Function to run the grid search with a progress bar and multiple threads
def grid_search_prophet(df, param_grid, periods=24, max_workers=4):
    results = []

    # Create all combinations of hyperparameters
    param_combinations = list(itertools.product(*param_grid.values()))

    # Use ThreadPoolExecutor for multi-threading
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks and create a progress bar
        future_to_params = {executor.submit(run_prophet_model, df, params, periods): params for params in param_combinations}

        # Use tqdm to display a progress bar
        for future in tqdm(as_completed(future_to_params), total=len(future_to_params), desc="Grid Search Progress"):
            params = future_to_params[future]
            try:
                result = future.result()
                if result:
                    results.append(result)
            except Exception as e:
                print(f"An error occurred for params {params}: {e}")

    # Return all results
    return results

In [8]:
logging.getLogger('cmdstanpy').setLevel(logging.CRITICAL)
logging.getLogger('prophet').setLevel(logging.CRITICAL)
os.environ['CMDSTANPY_NO_PROGRESS'] = '1'
os.environ['PROPHET_NO_PROGRESS_BAR'] = '1'

param_grid = {
    'changepoint_prior_scale': [0.01, 0.05, 0.1],
    'seasonality_mode': ['additive', 'multiplicative'],
    'seasonality_prior_scale': [5.0, 10.0, 15.0],
    'n_changepoints': [10, 25, 50],
    'yearly_seasonality': [True, False],
    'monthly_seasonality': [True, False],
    'uncertainty_samples': [500, 1000, 1500, 2000, 5000],
    'yearly_fourier_order': [5, 10, 20],
    'monthly_fourier_order': [3, 5, 10]
}

# Run the grid search
grid_search_results = grid_search_prophet(df, param_grid, periods=24, max_workers=10)

# Sort results by RMSE (lowest first)
sorted_results = sorted(grid_search_results, key=lambda x: x['rmse'])

# Show the best result
print(f"Best result: {sorted_results[0]}")

Grid Search Progress:   0%|          | 0/9720 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Best result: {'params': (0.01, 'multiplicative', 5.0, 25, True, False, 500, 5, 3), 'rmse': 1424.280696660563, 'mape': 0.31336719556620773, 'coverage': 0.6161440185830429}


In [9]:
print(f"Best result: {sorted_results[0]}")

Best result: {'params': (0.01, 'multiplicative', 5.0, 25, True, False, 500, 5, 3), 'rmse': 1424.280696660563, 'mape': 0.31336719556620773, 'coverage': 0.6161440185830429}
